In [1]:
import argparse
import random

import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
import learn2learn as l2l
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from ResNet import ResNet,Block 
import torchvision

In [2]:
train_path= "/home/logiceat3r/data-cancer/MIAS/train"

In [3]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1)
    acc = (predictions == targets).sum().float()
    acc /= len(targets)
    return acc.item()

In [11]:
lr=0.005
maml_lr=0.01
iterations=1000
ways=2
shots=5
tps=32
fas=5
device=torch.device("cpu")
download_location='/home/logiceat3r/data-cancer/MIAS/train'

In [12]:

transformations = transforms.Compose([
transforms.ToTensor(),
transforms.Resize((244,244)),
transforms.Normalize((0.1307,), (0.3081,)),
lambda x: x.view(3, 244, 244),
])
mias_train = l2l.data.MetaDataset(datasets.ImageFolder(train_path,transform=transformations))

In [13]:
train_tasks = l2l.data.TaskDataset(mias_train,
                               task_transforms=[
                                    l2l.data.transforms.LoadData(mias_train),
                                    l2l.data.transforms.NWays(mias_train, ways),
                               ],
                               num_tasks=2000)

train_tasks[0]

[tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]],
 
          [[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]],
 
          [[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-

In [14]:
model = ResNet(101, Block, 3, 1000)

In [15]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

In [28]:
meta_model = l2l.algorithms.MAML(model, lr=maml_lr)
opt = optim.Adam(meta_model.parameters(), lr=lr)
loss_func = nn.NLLLoss(reduction='mean')

In [29]:
for iteration in range(iterations):
    iteration_error = 0.0
    iteration_acc = 0.0
    for _ in range(tps):
        learner = meta_model.clone()
        train_task = train_tasks.sample()
        data, labels = train_task
        data = data.to(device)
        labels = labels.to(device)

        # Separate data into adaptation/evalutation sets
        adaptation_indices = np.zeros(data.size(0), dtype=bool)
        adaptation_indices[np.arange(shots*ways) * 2] = True
        evaluation_indices = torch.from_numpy(~adaptation_indices)
        adaptation_indices = torch.from_numpy(adaptation_indices)
        adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
        evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

        # Fast Adaptation
        for step in range(fas):
            train_error = loss_func(learner(adaptation_data), adaptation_labels)
            learner.adapt(train_error)

        # Compute validation loss
        predictions = learner(evaluation_data)
        valid_error = loss_func(predictions, evaluation_labels)
        valid_error /= len(evaluation_data)
        valid_accuracy = accuracy(predictions, evaluation_labels)
        iteration_error += valid_error
        iteration_acc += valid_accuracy

    iteration_error /= tps
    iteration_acc /= tps
    print('Loss : {:.3f} Acc : {:.3f}'.format(iteration_error.item(), iteration_acc))

    # Take the meta-learning step
    opt.zero_grad()
    iteration_error.backward()
    opt.step()


ValueError: Sample larger than population or is negative